In [15]:
# coding: utf-8

import numpy as np
import pandas as pd
import pickle
import gzip
from collections import Counter
from gensim.models import KeyedVectors


In [4]:
refdf_path = "refcoco_refdf_heads.json.gz"

In [35]:
refdf = pd.read_json(refdf_path,compression='gzip', orient='split')

In [36]:
refdf.head()

,i_corpus,image_id,region_id,names,cat,ix_Xfile
0,1,581857,1719310,"[lady, shirt, person]",1,34271
1,1,581857,463958,"[woman, person]",1,34270
2,1,581839,495152,[person],1,34771
3,1,581839,485695,"[girl, lady, woman, person]",1,34770
4,1,581789,453177,"[woman, person]",1,32032


In [32]:
refdf[refdf.region_id == 57115]

,i_corpus,image_id,region_id,names,cat
1675,1,563447,57115,[horse],19


In [33]:
refdf[refdf.image_id == 480023]

,i_corpus,image_id,region_id,names,cat
8905,1,480023,1745965,"[person, hand]",1
8906,1,480023,1292891,"[person, shirt]",1
8907,1,480023,498741,"[person, man, guy]",1
8908,1,480023,438029,"[person, girl, woman]",1


In [34]:
refdf[refdf.region_id == 448467]

,i_corpus,image_id,region_id,names,cat
7498,1,495875,448467,"[person, man, guy]",1


In [7]:
len(refdf)

50000

In [9]:
vocab = []
for n in refdf.names:
    vocab += list(n)

In [12]:
len(list(set(vocab)))

2901

In [19]:
"" in vocab

False

In [14]:
list(set(vocab))

['clump',
 'sculpture',
 'badass',
 'elks',
 'driver',
 'flowers',
 'lavender',
 'pancakes',
 'stand',
 'rabbit',
 'chunks',
 'oclock',
 'beam',
 'ber',
 'individual',
 'chic',
 'brixton',
 'club',
 'women',
 'sammy',
 'buss',
 'necklace',
 'eating',
 'model',
 'poerson',
 'flower',
 'blender',
 'banannas',
 'batte',
 'armhand',
 'strawberries',
 'delk',
 'hoddie',
 'universal',
 'rite',
 'garaffe',
 'xxx',
 'mascot',
 'character',
 'smack',
 'carpet',
 'kernel',
 'overalls',
 'lips',
 'macbook',
 'stuffed',
 'dials',
 'uniform',
 'see',
 'hor',
 'tables',
 'cranberry',
 'menu',
 'hunk',
 'hello',
 'kity',
 'penguin',
 'truckvan',
 'tractor',
 'decorative',
 'moo',
 'popcorn',
 'trash',
 'topping',
 'stick',
 'pepsi',
 'berries',
 'armrest',
 'sink',
 'outfit',
 'whitew',
 'creamy',
 'bear',
 'wannabe',
 'rght',
 'ho',
 'sort',
 'glare',
 'sandiwch',
 'vodka',
 'priest',
 'page',
 'gril',
 'plater',
 'listerine',
 'backrest',
 'geisha',
 'mcneff',
 'lounger',
 'metor',
 'plaid',
 'ok',

In [16]:
glove_model = KeyedVectors.load_word2vec_format("glove.6B/glove.6B.300d.w2v.bin")

In [18]:
glove_model['person'];

In [20]:
vocab = list(set(vocab))


In [21]:
glove_vocab = [ w for w in vocab if w in glove_model]

In [22]:
len(glove_vocab)

2393

In [25]:
print('person',' '.join([str(i) for i in glove_model['person']]))

person -0.55598 0.027967 -0.32289 -0.075042 -0.13179 0.13271 -0.15578 0.11468 -0.055989 -1.6266 0.12912 -0.20525 -0.2162 -0.087432 -0.085335 0.033107 -0.22467 -0.099248 -0.21777 -0.25671 0.063234 0.50175 -0.16002 0.30961 0.20912 -0.29045 0.042019 -0.022472 0.24542 -0.075772 -0.56552 0.62501 -0.35209 -0.60992 -0.93654 0.25625 0.21782 -0.24591 -0.53093 0.20034 0.022471 0.073258 -0.029808 0.082042 -0.26838 0.011861 0.5919 0.008274 -0.07756 0.37252 0.12574 0.05661 0.61548 0.19826 -0.053163 0.27715 0.15825 0.13941 0.35109 -0.21844 0.10418 0.089417 0.11362 0.25903 -0.10444 -0.30457 -0.36033 -0.22062 -0.47356 0.48017 0.037563 -0.42485 -0.026248 -0.14516 0.012868 0.041353 0.1957 0.34752 0.15859 0.097584 -0.1535 0.24506 -0.12282 0.24122 -0.42262 -0.20683 -0.26847 -0.22687 -0.038363 0.53815 -0.77009 0.3295 -0.42145 -0.29034 0.59593 -0.041209 -0.3399 0.3474 0.14502 -0.21154 -0.29105 0.00061433 -0.034839 -0.06446 0.50316 0.34938 0.61775 -0.15638 -0.001032 0.13306 -0.0085934 0.5276 0.22607 -0.19192

In [26]:
from gensim.scripts.glove2word2vec import glove2word2vec


In [27]:
glove2word2vec("glove.6B/glove.6B.300d.heads.txt", "glove.6B/glove.6B.300d.heads.w2v.bin")

(779, 300)